<a href="https://colab.research.google.com/github/olyavydrina/-/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22LTSM%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# Максимальное количество используемых слов 
num_words = 1000
# Максимальная длина новости
max_news_len = 500
# Количество классов новостей
nb_classes = 2

## Загрузка набора данных

Загружаем данные для обучения

In [ ]:
!wget https://github.com/olyavydrina/-/blob/main/train.csv -O train.csv

--2022-12-25 21:02:46--  https://github.com/olyavydrina/-/blob/main/train.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘train.csv’

train.csv               [ <=>                ]   2.77M  --.-KB/s    in 0.07s   

2022-12-25 21:02:47 (38.8 MB/s) - ‘train.csv’ saved [2908404]



Загружаем данные для тестирования

In [ ]:
!wget https://github.com/olyavydrina/-/blob/main/test.csv -O test.csv

--2022-12-25 21:03:01--  https://github.com/olyavydrina/-/blob/main/test.csv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘test.csv’

test.csv                [ <=>                ] 974.48K  --.-KB/s    in 0.03s   

2022-12-25 21:03:01 (27.5 MB/s) - ‘test.csv’ saved [997865]



Загружаем имена классов

In [ ]:
!wget https://github.com/olyavydrina/-/blob/main/classes.txt -O classes.txt

--2022-12-25 21:06:59--  https://github.com/olyavydrina/-/blob/main/classes.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘classes.txt’

classes.txt             [ <=>                ] 140.88K  --.-KB/s    in 0.01s   

2022-12-25 21:07:00 (12.3 MB/s) - ‘classes.txt’ saved [144256]



## Просматриваем данные

In [ ]:
!ls

classes.txt  sample_data  test.csv  train.csv


In [ ]:
!cat classes.txt








<!DOCTYPE html>
<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">
  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">



  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-719f1193e0c0.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-0c343b529849.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" media="all" rel="stylesheet" data-href="https://github

In [ ]:
!head train.csv








<!DOCTYPE html>
<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">
  <head>


In [ ]:
!head test.csv








<!DOCTYPE html>
<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">
  <head>


In [ ]:
!wc -l train.csv
!wc -l test.csv

32943 train.csv
10686 test.csv


## Загружаем данные в память

Читаем данные из файла

In [ ]:
train = pd.read_csv('train.csv', 
                    header=None, 
                    names=['class', 'title', 'text', 'link'])

In [ ]:
train

,class,title,text,link
0,<!DOCTYPE html>,NaN,NaN,NaN
1,"<html lang=""en"" data-color-mode=""auto"" data-li...",NaN,NaN,NaN
2,<head>,NaN,NaN,NaN
3,"<meta charset=""utf-8"">",NaN,NaN,NaN
4,"<link rel=""dns-prefetch"" href=""https://githu...",NaN,NaN,NaN
...,...,...,...,...
32766,</template>,NaN,NaN,NaN
32767,</div>,NaN,NaN,NaN
32768,"<div id=""js-global-screen-reader-notice"" c...",NaN,NaN,NaN
32769,</body>,NaN,NaN,NaN


Выделяем данные для обучения

In [ ]:
news = train['title'] + " " + train['text']

In [ ]:
news[:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
dtype: object

Выделяем правильные ответы

In [ ]:
y_train = utils.to_categorical(train['class'] - 1, nb_classes)

TypeError: ignored

In [ ]:
y_train

NameError: ignored

## Токенизация текста

In [ ]:
news[:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
dtype: object

Создаем токенизатор Keras

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

Обучаем токенизатор на новостях

In [ ]:
tokenizer.fit_on_texts(news)

AttributeError: ignored

Просматриваем словарь токенизатора

In [ ]:
tokenizer.word_index

{}

Преобразуем новости в числовое представление

In [ ]:
sequences = tokenizer.texts_to_sequences(news)

AttributeError: ignored

Просматриваем новости в числовом представлении

In [ ]:
index = 1
print(news[index])
print(sequences[index])

nan


NameError: ignored

In [ ]:
tokenizer.word_index['war']

KeyError: ignored

Ограничиваем длину названия и самой статьи

In [ ]:
x_train = pad_sequences(sequences, maxlen=max_news_len)

NameError: ignored

In [ ]:
x_train[:5]

NameError: ignored

## Cеть LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(Embedding(num_words, 32, input_length=max_news_len))
model_lstm.add(LSTM(16))
model_lstm.add(Dense(2, activation='sigmoid'))

In [ ]:
model_lstm.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           32000     
                                                                 
 lstm (LSTM)                 (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 2)                 34        
                                                                 
Total params: 35,170
Trainable params: 35,170
Non-trainable params: 0
_________________________________________________________________


Создаем callback для сохранения нейронной сети на каждой эпохе, если качество работы на проверочном наборе данных улучшилось. Сеть сохраняется в файл `best_model.h5`

In [ ]:
model_lstm_save_path = 'best_model_lstm.h5'
checkpoint_callback_lstm = ModelCheckpoint(model_lstm_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history_lstm = model_lstm.fit(x_train, 
                              y_train, 
                              epochs=5,
                              batch_size=128,
                              validation_split=0.1,
                              callbacks=[checkpoint_callback_lstm])

NameError: ignored

In [ ]:
plt.plot(history_lstm.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history_lstm.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

NameError: ignored

## Загружаем набор данных для тестирования

In [ ]:
test = pd.read_csv('test.csv', 
                    header=None, 
                    names=['class', 'title', 'text', 'link'])

In [ ]:
test

,class,title,text,link
0,<!DOCTYPE html>,NaN,NaN,NaN
1,"<html lang=""en"" data-color-mode=""auto"" data-li...",NaN,NaN,NaN
2,<head>,NaN,NaN,NaN
3,"<meta charset=""utf-8"">",NaN,NaN,NaN
4,"<link rel=""dns-prefetch"" href=""https://githu...",NaN,NaN,NaN
...,...,...,...,...
10311,</template>,NaN,NaN,NaN
10312,</div>,NaN,NaN,NaN
10313,"<div id=""js-global-screen-reader-notice"" c...",NaN,NaN,NaN
10314,</body>,NaN,NaN,NaN


Преобразуем новости в числовое представление

Используется токенизатор, обученный на наборе данных train.

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['text'])

AttributeError: ignored

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max_news_len)

NameError: ignored

In [ ]:
x_test[:5]

NameError: ignored

Правильные ответы

In [ ]:
y_test = utils.to_categorical(test['class'] - 1, nb_classes)

TypeError: ignored

In [ ]:
y_test

NameError: ignored

## Оцениваем качество работы сети на тестовом наборе данных

### Сеть LSTM

In [ ]:
model_lstm.load_weights(model_lstm_save_path)

OSError: ignored

In [ ]:
model_lstm.evaluate(x_test, y_test, verbose=1)

NameError: ignored